Deep Learning
=============

Assignment 2
------------

Previously in `1_notmnist.ipynb`, we created a pickle with formatted datasets for training, development and testing on the [notMNIST dataset](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html).

The goal of this assignment is to progressively train deeper and more accurate models using TensorFlow.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (32000, 28, 28) (32000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (32000, 784) (32000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


We're first going to train a multinomial logistic regression using simple gradient descent.

TensorFlow works like this:
* First you describe the computation that you want to see performed: what the inputs, the variables, and the operations look like. These get created as nodes over a computation graph. This description is all contained within the block below:

      with graph.as_default():
          ...

* Then you can run the operations on this graph as many times as you want by calling `session.run()`, providing it outputs to fetch from the graph that get returned. This runtime operation is all contained in the block below:

      with tf.Session(graph=graph) as session:
          ...

Let's load all the data into TensorFlow and build the computation graph corresponding to our training:

In [4]:
# REF: https://www.tensorflow.org/get_started/summaries_and_tensorboard
def variable_summaries(var):
  """Attach a lot of summaries to a Tensor (for TensorBoard visualization)."""
  with tf.name_scope('summaries'):
    mean = tf.reduce_mean(var)
    tf.summary.scalar('mean', mean)
    with tf.name_scope('stddev'):
      stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
    tf.summary.scalar('stddev', stddev)
    tf.summary.scalar('max', tf.reduce_max(var))
    tf.summary.scalar('min', tf.reduce_min(var))
    tf.summary.histogram('histogram', var)

In [21]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = np.random.choice(np.arange(len(train_dataset)),10000)
learning_rate = .5

graph1 = tf.Graph()
with graph1.as_default():
  tf.summary.scalar('image_size', image_size)
  tf.summary.scalar('num_labels', num_labels)
  tf.summary.scalar('learning_rate', learning_rate)

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  with tf.name_scope('data'):
    with tf.name_scope('train'):
      tf_train_dataset = tf.constant(train_dataset[train_subset, :])
      tf_train_labels = tf.constant(train_labels[train_subset])
    with tf.name_scope('valid'):
      tf_valid_dataset = tf.constant(valid_dataset)
    with tf.name_scope('test'):
      tf_test_dataset = tf.constant(test_dataset)
  
  with tf.name_scope('ff1'):
    # Variables.
    # These are the parameters that we are going to be training. The weight
    # matrix will be initialized using random values following a (truncated)
    # normal distribution. The biases get initialized to zero.
    with tf.name_scope('weights'):
        weights = tf.Variable(
            tf.truncated_normal([image_size * image_size, num_labels]))
        variable_summaries(weights)
    with tf.name_scope('biases'):
        biases = tf.Variable(tf.zeros([num_labels]))
        variable_summaries(biases)
        
    # Training computation.
    # We multiply the inputs with the weight matrix, and add biases. We compute
    # the softmax and cross-entropy (it's one operation in TensorFlow, because
    # it's very common, and it can be optimized). We take the average of this
    # cross-entropy across all training examples: that's our loss.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    variable_summaries(logits)
  
  with tf.name_scope('error'):
    diff = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels)
    loss = tf.reduce_mean(diff)
    tf.summary.scalar('cross_entropy', loss)
    
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  with tf.name_scope('opt'):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  with tf.name_scope('out'):
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Let's run this computation and iterate:

In [22]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [23]:
def run_graph(graph, num_steps = 1001, summary_path = './summary/'):
    with tf.Session(graph=graph) as session:
        # This is a one-time operation which ensures the parameters get initialized as
        # we described in the graph: random weights for the matrix, zeros for the
        # biases.

        # Merge all the summaries and write them out to /tmp/mnist_logs (by default)
        merged_summary = tf.summary.merge_all()
        train_writer = tf.summary.FileWriter(summary_path+'train', graph=graph)
        test_writer = tf.summary.FileWriter(summary_path+'test')
        tf.global_variables_initializer().run()
        print('Initialized')

        def test_step(step):
            summary, test = session.run([merged_summary, test_prediction])
            test_writer.add_summary(summary, step)
            test = accuracy(test, test_labels)
            print("Step {}: Test: {}%".format(step, test))
            return test

        def train_step(step, verify=False):
            if verify:
                summary, _, train, valid = session.run([merged_summary, optimizer, train_prediction, valid_prediction])
                train_writer.add_summary(summary, step)
                train = accuracy(train, train_labels[train_subset])
                valid = accuracy(valid, valid_labels)
                print("Step {}: Train: {}% Valid: {}%".format(step, train, valid))
                return train, valid
            else:
                summary, _ = session.run([merged_summary, optimizer])
                train_writer.add_summary(summary, step)

        # Early Stopping
        saver = tf.train.Saver([weights, biases])
        stopping_threshold = 3

        # Main Loop
        for step in range(num_steps):
            # Run the computations. We tell .run() that we want to run the optimizer,
            # and get the loss value and the training predictions returned as numpy
            # arrays.
            # Every 1000 steps, test the test accuracy without training
            if (step == num_steps):
                test_step(step)

            # Every 100 steps, test training and validation accuracy
            elif (step % 100 == 0):
                saver.save(session, summary_path+"train/single_layer{}.chk".format(step))
                train, valid = train_step(step, verify=True)

                # Early Stopping
                if abs(train-valid)>stopping_threshold:
                    print("Stopping Early!")
                    test_step(step)
                    break

            # Normal, fast, training step
            else:
                train_step(step)

In [24]:
run_graph(graph1, summary_path="./summary/fullyconnected/singleconnected/")

Initialized
Step 0: Train: 8.76% Valid: 8.32%
Step 100: Train: 71.16% Valid: 69.83%
Step 200: Train: 74.25% Valid: 72.2%
Step 300: Train: 75.87% Valid: 73.11%
Step 400: Train: 76.91% Valid: 73.48%
Stopping Early!
Step 400: Test: 80.63%


Let's now switch to stochastic gradient descent training instead, which is much faster.

The graph will be similar, except that instead of holding all the training data into a constant node, we create a `Placeholder` node which will be fed actual data at every call of `session.run()`.

In [55]:
batch_size = 128

graph2 = tf.Graph()
with graph2.as_default():
  tf.summary.scalar('image_size', image_size)
  tf.summary.scalar('batch_size', batch_size)
  tf.summary.scalar('num_labels', num_labels)
  tf.summary.scalar('learning_rate', learning_rate)

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  with tf.name_scope('data'):
    with tf.name_scope('train'):
      tf_train_dataset = tf.placeholder(tf.float32,
                                        shape=(batch_size, image_size * image_size))
      tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    with tf.name_scope('valid'):
      tf_valid_dataset = tf.constant(valid_dataset)
    with tf.name_scope('test'):
      tf_test_dataset = tf.constant(test_dataset)
  
  with tf.name_scope('ff1'):
    # Variables.
    # These are the parameters that we are going to be training. The weight
    # matrix will be initialized using random values following a (truncated)
    # normal distribution. The biases get initialized to zero.
    with tf.name_scope('weights'):
        weights = tf.Variable(
            tf.truncated_normal([image_size * image_size, num_labels]))
        variable_summaries(weights)
    with tf.name_scope('biases'):
        biases = tf.Variable(tf.zeros([num_labels]))
        variable_summaries(biases)
        
    # Training computation.
    # We multiply the inputs with the weight matrix, and add biases. We compute
    # the softmax and cross-entropy (it's one operation in TensorFlow, because
    # it's very common, and it can be optimized). We take the average of this
    # cross-entropy across all training examples: that's our loss.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    variable_summaries(logits)
  
  with tf.name_scope('error'):
    diff = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels)
    loss = tf.reduce_mean(diff)
    tf.summary.scalar('cross_entropy', loss)
    
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  with tf.name_scope('opt'):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  with tf.name_scope('out'):
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Let's run it:

In [56]:
def run_batch_graph(graph, num_steps = 1001, summary_path = './summary/', save_layers=[weights, biases]):
    with tf.Session(graph=graph) as session:
        # Merge all the summaries and write them out to /tmp/mnist_logs (by default)
        merged_summary = tf.summary.merge_all()
        train_writer = tf.summary.FileWriter(summary_path+'train', graph=graph)
        test_writer = tf.summary.FileWriter(summary_path+'test')
        tf.global_variables_initializer().run()
        print('Initialized')

        def get_feed_dict(step):
            # Pick an offset within the training data, which has been randomized.
            # Note: we could use better randomization across epochs.
            offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
            # Generate a minibatch.
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]
            # Prepare a dictionary telling the session where to feed the minibatch.
            # The key of the dictionary is the placeholder node of the graph to be fed,
            # and the value is the numpy array to feed to it.
            feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
            return feed_dict, (batch_data, batch_labels)

        def test_step(step):
            feed_dict, (batch_data, batch_labels) = get_feed_dict(step)
            summary, test = session.run([merged_summary, test_prediction], feed_dict=feed_dict)
            test_writer.add_summary(summary, step)
            test = accuracy(test, test_labels)
            print("Step {}: Test: {}%".format(step, test))
            return test

        def train_step(step, verify=False):
            feed_dict, (batch_data, batch_labels) = get_feed_dict(step)
            if verify:
                summary, _, train, valid = session.run([merged_summary, optimizer, train_prediction, valid_prediction], feed_dict=feed_dict)
                train_writer.add_summary(summary, step)
                train = accuracy(train, batch_labels)
                valid = accuracy(valid, valid_labels)
                print("Step {}: Train: {}% Valid: {}%".format(step, train, valid))
                return train, valid
            else:
                summary, _ = session.run([merged_summary, optimizer], feed_dict=feed_dict)
                train_writer.add_summary(summary, step)

        # Early Stopping
        saver = tf.train.Saver(save_layers)
        earliest_stop = 300
        stopping_threshold = 3

        # Main Loop
        for step in range(num_steps):
            # Run the computations. We tell .run() that we want to run the optimizer,
            # and get the loss value and the training predictions returned as numpy
            # arrays.
            # Every 1000 steps, test the test accuracy without training
            if (step == num_steps):
                test_step(step)

            # Every 100 steps, test training and validation accuracy
            elif (step % 100 == 0):
                saver.save(session, summary_path+"train/single_layer{}.chk".format(step))
                train, valid = train_step(step, verify=True)

                # Early Stopping
                if abs(train-valid)>stopping_threshold and step>earliest_stop:
                    print("Stopping Early!")
                    test_step(step)
                    break

            # Normal, fast, training step
            else:
                train_step(step)

In [57]:
run_batch_graph(graph2, summary_path="./summary/fullyconnected/batches/")

Initialized
Step 0: Train: 15.625% Valid: 14.43%
Step 100: Train: 73.4375% Valid: 68.18%
Step 200: Train: 74.21875% Valid: 72.61%
Step 300: Train: 71.09375% Valid: 73.57%
Step 400: Train: 75.0% Valid: 74.4%
Step 500: Train: 71.09375% Valid: 74.92%
Stopping Early!
Step 500: Test: 82.56%


---
Problem
-------

Turn the logistic regression example with SGD into a 1-hidden layer neural network with rectified linear units [nn.relu()](https://www.tensorflow.org/versions/r0.7/api_docs/python/nn.html#relu) and 1024 hidden nodes. This model should improve your validation / test accuracy.

---

In [67]:
batch_size = 128
hidden_size = 1024

graph3 = tf.Graph()
with graph3.as_default():
  tf.summary.scalar('image_size', image_size)
  tf.summary.scalar('batch_size', batch_size)
  tf.summary.scalar('num_labels', num_labels)
  tf.summary.scalar('num_labels', hidden_size)
  tf.summary.scalar('learning_rate', learning_rate)

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  with tf.name_scope('data'):
    with tf.name_scope('train'):
      tf_train_dataset = tf.placeholder(tf.float32,
                                        shape=(batch_size, image_size * image_size))
      tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    with tf.name_scope('valid'):
      tf_valid_dataset = tf.constant(valid_dataset)
    with tf.name_scope('test'):
      tf_test_dataset = tf.constant(test_dataset)
  
  # Feed Forward Layer 1
  with tf.name_scope('ff1'):
    with tf.name_scope('weights'):
        weights1 = tf.Variable(
            tf.truncated_normal([image_size * image_size, hidden_size]))
        variable_summaries(weights1)
    with tf.name_scope('biases'):
        biases1 = tf.Variable(tf.zeros([hidden_size]))
        variable_summaries(biases1)
    with tf.name_scope('linear'):
        logits1 = tf.matmul(tf_train_dataset, weights1) + biases1
        variable_summaries(logits1)
    with tf.name_scope('activation'):
        act1 = tf.nn.relu(logits1)
        variable_summaries(act1)
    
  # Feed Forward Layer 1
  with tf.name_scope('ff2'):
    with tf.name_scope('weights'):
        weights2 = tf.Variable(
            tf.truncated_normal([hidden_size, num_labels]))
        variable_summaries(weights2)
    with tf.name_scope('biases'):
        biases2 = tf.Variable(tf.zeros([num_labels]))
        variable_summaries(biases2)
    with tf.name_scope('linear'):
        logits2 = tf.matmul(act1, weights2) + biases2
        variable_summaries(logits2)
  
  with tf.name_scope('error'):
    diff = tf.nn.softmax_cross_entropy_with_logits(logits=logits2, labels=tf_train_labels)
    loss = tf.reduce_mean(diff)
    tf.summary.scalar('cross_entropy', loss)
    
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  with tf.name_scope('opt'):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  with tf.name_scope('out'):
    train_prediction = tf.nn.softmax(logits2)
    valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1), weights2) + biases2)
    test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1), weights2) + biases2)

In [68]:
run_batch_graph(graph3, summary_path="./summary/fullyconnected/hidden/", save_layers=[weights1, biases1, weights2, biases2])

Initialized
Step 0: Train: 10.15625% Valid: 19.8%
Step 100: Train: 82.8125% Valid: 76.63%
Step 200: Train: 80.46875% Valid: 80.29%
Step 300: Train: 80.46875% Valid: 79.64%
Step 400: Train: 83.59375% Valid: 80.51%
Stopping Early!
Step 400: Test: 87.93%
